In [8]:
import os
import pickle
import json
import random
import numpy as np
!pip install torch
import torch
import torch.nn as nn
import torch.nn.functional as F
!pip install torch_geometric -q
import torch_geometric
from torch_geometric.data import Data

In [9]:
# set seed
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)

In [10]:
import warnings
warnings.filterwarnings("ignore")

DATA_PATH = "done"

In [11]:
import os

def get_files_in_folder(folder_path):
    file_list = []
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        if os.path.isfile(file_path):
            file_list.append(file_path)
    return file_list

# Replace 'folder_path' with the path to the folder you want to read files from
folder_path = "done"
verilog_files = get_files_in_folder(folder_path)
print(verilog_files)


['done\\adder11.txt', 'done\\adder12.txt', 'done\\adder13.txt', 'done\\adder14.txt', 'done\\adder15.txt', 'done\\adder16.txt', 'done\\adder17.txt', 'done\\adder18.txt', 'done\\adder19.txt', 'done\\adder2.txt', 'done\\adder20.txt', 'done\\adder5.txt', 'done\\adder6.txt', 'done\\adder8.txt', 'done\\ALU10.txt', 'done\\ALU13.txt', 'done\\ALU14.txt', 'done\\ALU15.txt', 'done\\ALU2.txt', 'done\\ALU6.txt', 'done\\ALU7.txt', 'done\\ALU8.txt', 'done\\ALU9.txt', 'done\\and1.txt', 'done\\and10.txt', 'done\\and12.txt', 'done\\and13.txt', 'done\\and14.txt', 'done\\and15.txt', 'done\\and16.txt', 'done\\and17.txt', 'done\\and18.txt', 'done\\and19.txt', 'done\\and2.txt', 'done\\and20.txt', 'done\\and21.txt', 'done\\and23.txt', 'done\\and25.txt', 'done\\and26.txt', 'done\\and27.txt', 'done\\and28.txt', 'done\\and29.txt', 'done\\and3.txt', 'done\\and30.txt', 'done\\and6.txt', 'done\\and7.txt', 'done\\and8.txt', 'done\\and9.txt', 'done\\comparator1.txt', 'done\\comparator13.txt', 'done\\comparator14.txt'

In [12]:
failed_files = []
for v in verilog_files:
    with open(v, "r") as file:
        loaded_data = json.load(file)
        # print(loaded_data[0])
        
        for node in loaded_data[0]:
            if len(node) != 4:
                failed_files.append(v)
                break
            
#how to save list in a txt file
with open("failed_files.txt", "w") as output:
    output.write(str(failed_files))

In [13]:
def extracting_attributes(verilog_file):
    try:
        if os.path.isfile(verilog_file):
            with open(verilog_file, "r") as file:
                loaded_data = json.load(file)
                nodes = loaded_data[0]
                edges = loaded_data[1]
                edge_atr = loaded_data[2]
                label = loaded_data[3]
                
                x = torch.tensor(nodes, dtype=torch.long)
                edge_index = torch.tensor(edges, dtype=torch.long)
                # edge_atr = torch.tensor(edge_atr, dtype=torch.long)
                y = torch.tensor(label, dtype=torch.long)
                num_nodes = x.size(0)
                # print(num_nodes)
                # Create batch assignment vector (assuming one graph per file)
                batch = torch.zeros(num_nodes, dtype=torch.long)
                data = Data(x=x, edge_index=edge_index, y = y, batch = batch)
                return data
    
    except Exception as e:
        print(e)
        return e

# temp=extracting_attributes("./done/adder6.txt")
# temp

In [14]:
import os
import json
import torch
from torch.utils.data import DataLoader, Dataset

class VerilogDataset(Dataset):  # Using Dataset from torch_geometric
    def __init__(self, verilog_files):
        print(f"Loaded {len(verilog_files)} Verilog files.")
        self.verilog_files = verilog_files

    def __len__(self):
        return len(self.verilog_files)

    def __getitem__(self, idx):
        verilog_file = self.verilog_files[idx]
        data = extracting_attributes(verilog_file)
        return data

dataset = VerilogDataset(verilog_files)
print(len(dataset))

Loaded 405 Verilog files.
405


In [15]:
temp = dataset[0]
temp

Data(x=[165, 4], edge_index=[2, 208], y=[1], batch=[165])

In [17]:
def graph_stat(dataset, verilog_files):
    """
    TODO: calculate the statistics of the ENZYMES dataset.
    
    Outputs:
        min_num_nodes: min number of nodes
        max_num_nodes: max number of nodes
        mean_num_nodes: average number of nodes
        min_num_edges: min number of edges
        max_num_edges: max number of edges
        mean_num_edges: average number of edges
    """
    # for ind,data in enumerate(dataset):
        # print(verilog_files[ind])
        # print(data)
        # print(len(data.x[1]))
        
    nodes_edges = [(data.num_nodes, data.num_edges) for data in dataset]
    num_nodes, num_edges = list(list(zip(*nodes_edges))[0]), list(list(zip(*nodes_edges))[1])
    min_num_nodes = min(num_nodes)
    max_num_nodes = max(num_nodes)
    mean_num_nodes = np.mean(num_nodes)
    min_num_edges = min(num_edges)
    max_num_edges = max(num_edges)
    mean_num_edges = np.mean(num_edges)
    
    print(f"min_num_nodes: {min_num_nodes}")
    print(f"max_num_nodes: {max_num_nodes}")
    print(f"mean_num_nodes: {mean_num_nodes}")
    print(f"min_num_edges: {min_num_edges}")
    print(f"max_num_edges: {max_num_edges}")
    print(f"mean_num_edges: {mean_num_edges}")

graph_stat(dataset, verilog_files)

min_num_nodes: 3
max_num_nodes: 165
mean_num_nodes: 15.59753086419753
min_num_edges: 2
max_num_edges: 208
mean_num_edges: 19.409876543209876


In [28]:
from torch_geometric.data import Data
from torch.utils.data.dataloader import default_collate

def custom_collate(batch):
    if isinstance(batch[0], Data):
        return batch
    else:
        return default_collate(batch)

In [29]:
# from torch.utils.data import random_split

# # Define the sizes of training, validation, and test sets
# train_size = int(0.7 * len(dataset))  # 70% of the data for training
# val_size = int(0.15 * len(dataset))   # 15% of the data for validation
# test_size = len(dataset) - train_size - val_size  # Remaining data for testing

# # Split the dataset into training, validation, and test sets
# train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# # Create DataLoader for each set
# train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
# val_dataloader = DataLoader(val_dataset, batch_size=32)
# test_dataloader = DataLoader(test_dataset, batch_size=32)


from torch.utils.data import random_split

# Define the size of the training set (e.g., 70% of the data)
train_size = int(0.7 * len(dataset))

# Calculate the size of the testing set
test_size = len(dataset) - train_size

# Split the dataset into training and testing sets
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Create DataLoader for each set
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=custom_collate)
test_loader = DataLoader(test_dataset, batch_size=32, collate_fn=custom_collate)


In [30]:
len(train_loader.dataset)

283

In [31]:
loader_iter = iter(train_loader)
batch = next(loader_iter)
print(batch)
# print(batch.num_graphs)

14
5
10
24
3
expected sequence of length 4 at dim 1 (got 3)
43
11
5
38
4
16
5
4
9
7
18
31
7
19
15
8
expected sequence of length 4 at dim 1 (got 3)
35
4
4
3
7
10
4
8
12
[Data(x=[14, 4], edge_index=[2, 13], edge_attr=[13], y=[1], batch=[14]), Data(x=[5, 4], edge_index=[2, 4], edge_attr=[4], y=[1], batch=[5]), Data(x=[10, 4], edge_index=[2, 10], edge_attr=[10], y=[1], batch=[10]), Data(x=[24, 4], edge_index=[2, 27], edge_attr=[27], y=[1], batch=[24]), Data(x=[3, 4], edge_index=[2, 2], edge_attr=[2], y=[1], batch=[3]), ValueError('expected sequence of length 4 at dim 1 (got 3)'), Data(x=[43, 4], edge_index=[2, 51], edge_attr=[51], y=[1], batch=[43]), Data(x=[11, 4], edge_index=[2, 18], edge_attr=[18], y=[1], batch=[11]), Data(x=[5, 4], edge_index=[2, 4], edge_attr=[4], y=[1], batch=[5]), Data(x=[38, 4], edge_index=[2, 44], edge_attr=[44], y=[1], batch=[38]), Data(x=[4, 4], edge_index=[2, 4], edge_attr=[4], y=[1], batch=[4]), Data(x=[16, 4], edge_index=[2, 19], edge_attr=[19], y=[1], batch=

In [32]:
for data in train_loader:
    print(len(data))
    # print(data.num_graphs)
    break

7
48
4
52
3
16
7
17
15
6
27
4
38
48
16
4
8
21
12
11
31
15
18
3
7
5
4
9
11
expected sequence of length 4 at dim 1 (got 3)
4
51
32


In [33]:
import math
from torch_geometric.utils import to_dense_adj, add_self_loops

class GCNConv(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GCNConv, self).__init__()
        self.theta = nn.Parameter(torch.FloatTensor(in_channels, out_channels))
        # Initialize the parameters.
        stdv = 1. / math.sqrt(out_channels)
        self.theta.data.uniform_(-stdv, stdv)
    
    def forward(self, x, edge_index):
        """
        TODO:
            1. Generate the adjacency matrix with self-loop \hat{A} using edge_index.
            2. Calculate the diagonal degree matrix \hat{D}.
            3. Calculate the output X' with torch.mm using the equation above.
        """

        num_nodes = x.shape[0]
        A = torch.sparse_coo_tensor(edge_index, torch.ones(edge_index.shape[1]), (num_nodes, num_nodes))
        A = A.to_dense()
        A_hat = A + torch.eye(num_nodes)
        
        A_sum = torch.sum(A_hat, dim=1)
        D = torch.pow(A_sum, -0.5)
        D[D == float('inf')] = 0.0
        D_hat_sqrt = torch.diag(D)
        
        first = torch.mm(torch.mm(D_hat_sqrt, A_hat), D_hat_sqrt)
        second = torch.mm(x, self.theta)
        
        ret = torch.mm(first, second)
        
        return ret

In [34]:
# from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool

class GCN(torch.nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        """
        TODO:
            1. Define the first convolution layer using `GCNConv()`. Set `out_channels` to 64;
            2. Define the first activation layer using `nn.ReLU()`;
            3. Define the second convolution layer using `GCNConv()`. Set `out_channels` to 64;
            4. Define the second activation layer using `nn.ReLU()`;
            5. Define the third convolution layer using `GCNConv()`. Set `out_channels` to 64;
            6. Define the dropout layer using `nn.Dropout()`;
            7. Define the linear layer using `nn.Linear()`. Set `output_size` to 2.

        Note that for MUTAG dataset, the number of node features is 7, and the number of classes is 2.

        """
        
        self.gcn1 = GCNConv(in_channels=3, out_channels=64)
        self.a1 = nn.ReLU()
        self.gcn2 = GCNConv(in_channels=64, out_channels=64)
        self.a2 = nn.ReLU()
        self.gcn3 = GCNConv(in_channels=64, out_channels=64)
        self.dropout = nn.Dropout(p=0.4)
        self.linear = nn.Linear(in_features=64, out_features=18)

    def forward(self, x, edge_index, batch):
        """
        TODO:
            1. Pass the data through the frst convolution layer;
            2. Pass the data through the activation layer;
            3. Pass the data through the second convolution layer;
            4. Obtain the graph embeddings using the readout layer with `global_mean_pool()`;
            5. Pass the graph embeddgins through the dropout layer;
            6. Pass the graph embeddings through the linear layer.
            
        Arguments:
            x: [num_nodes, 3], node features
            edge_index: [2, num_edges], edges
            batch: [num_nodes], batch assignment vector which maps each node to its 
                   respective graph in the batch

        Outputs:
            probs: probabilities of shape (batch_size, 18)
        """
        x = self.gcn1(x, edge_index)
        x = self.a1(x)
        x = self.gcn2(x, edge_index)
        x = self.a2(x)
        x = self.gcn3(x, edge_index)
        x = global_mean_pool(x, batch)
        x = self.dropout(x)
        x = self.linear(x)
        
        probs = torch.nn.functional.softmax(x, dim=-1)
        
        return probs
        
        
        
GCN()

GCN(
  (gcn1): GCNConv()
  (a1): ReLU()
  (gcn2): GCNConv()
  (a2): ReLU()
  (gcn3): GCNConv()
  (dropout): Dropout(p=0.4, inplace=False)
  (linear): Linear(in_features=64, out_features=18, bias=True)
)

In [35]:
gcn = GCN()

# optimizer
optimizer = torch.optim.Adam(gcn.parameters(), lr=0.01)
# loss
criterion = torch.nn.CrossEntropyLoss()

def train(train_loader):
    gcn.train()
    for batch_data in train_loader:  # Iterate in batches over the training dataset.
        for data in batch_data:
            # print(data.x, data.edge_index, data.batch, data.y)
            out = gcn(data.x, data.edge_index, data.batch)
            loss = criterion(out, data.y)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        
        
def test(loader):
    gcn.eval()
    correct = 0
    for batch_data in loader:  # Iterate in batches over the training/test dataset.
        for data in batch_data:
            # print(data.x, data.edge_index, data.batch, data.y)
            out = gcn(data.x, data.edge_index, data.batch)  
            pred = out.argmax(dim=1)  # Use the class with highest probability.
            print(pred)
            print(data.y)
            print((pred == data.y))
            correct += int((pred == data.y).sum())  # Check against ground-truth labels.
    return correct / len(loader.dataset)  # Derive ratio of correct predictions.

import time

# Start the timer
start_time = time.time()

# Your training code here
for epoch in range(200):
    train(train_loader)
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch + 1:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')
    



# End the timer
end_time = time.time()

# Calculate the duration
duration = end_time - start_time
print("Training duration:", duration, "seconds")

20
16
expected sequence of length 4 at dim 1 (got 3)
48
9
5
15
4
5
7
7
expected sequence of length 4 at dim 1 (got 3)
5
11
4
7
16
8
expected sequence of length 4 at dim 1 (got 3)
7
expected sequence of length 4 at dim 1 (got 3)
19
8
43
52
5
6
expected sequence of length 4 at dim 1 (got 3)
7
22
17
21


RuntimeError: mat1 and mat2 shapes cannot be multiplied (20x4 and 3x64)

In [414]:
z = torch.tensor(2, dtype=torch.long)
e = torch.tensor(2, dtype=torch.long)

correct= int((z == e).sum())
correct

1

In [418]:
test_acc = test(test_loader)


tensor([17])
tensor([17])
tensor([True])
tensor([17])
tensor([10])
tensor([False])
tensor([17])
tensor([7])
tensor([False])
tensor([17])
tensor([17])
tensor([True])
tensor([5])
tensor([13])
tensor([False])
tensor([8])
tensor([4])
tensor([False])
tensor([13])
tensor([6])
tensor([False])
tensor([8])
tensor([2])
tensor([False])
tensor([17])
tensor([12])
tensor([False])
tensor([17])
tensor([2])
tensor([False])
tensor([13])
tensor([12])
tensor([False])
tensor([8])
tensor([1])
tensor([False])
tensor([13])
tensor([6])
tensor([False])
tensor([17])
tensor([17])
tensor([True])
tensor([13])
tensor([8])
tensor([False])
tensor([5])
tensor([5])
tensor([True])
tensor([17])
tensor([12])
tensor([False])
tensor([17])
tensor([10])
tensor([False])
tensor([17])
tensor([16])
tensor([False])
tensor([13])
tensor([4])
tensor([False])
tensor([8])
tensor([4])
tensor([False])
tensor([17])
tensor([17])
tensor([True])
tensor([17])
tensor([16])
tensor([False])
tensor([17])
tensor([17])
tensor([True])
tensor([5])
ten

In [419]:
test_acc

0.1958762886597938

In [425]:
torch.save(gcn.state_dict(), 'gcn_model.pth')

In [429]:
x = torch.tensor([[0,0,1], [0,0,1], [1,0,1], [3,3,1]], dtype=torch.float)
y = torch.tensor([2], dtype=torch.long)
edge_index = torch.tensor([[0, 1, 2],
                           [3, 3, 3]], dtype=torch.long)
gcn.eval()
batch = torch.tensor([0, 0, 0, 0], dtype=torch.long)
data_trial = Data(x=x, edge_index=edge_index, y=y, batch = batch)

out = gcn(data_trial.x, data_trial.edge_index, data_trial.batch)
pred = out.argmax(dim=1)
print(pred)

tensor([17])


In [ ]:
# gcn = GCN()
# gcn.load_state_dict(torch.load('gcn_model.pth'))

# Experiment 1


### Mode of Operation

* 321 verilog files 
* only 3 features             [type, operation_type, num_of_connections]
* no edge attribute
* 18 classes 
* 200 epochs 
* learning rate = 0.01
* Dropoout = 0.4
* Adam Optimizer
* train 70, test 30 (on whole dataset, not each class)
* time of training = seconds


### Results

* Train acc:  0.2902
* Test Acc: 0.1959


### Suggested Modifications for upcoming experiments

1) Clean dataset (by removing unnecessay, uninformative or wrong code files)
2) remove reduntant parsing (different files but same parsing)
3) include more informative features
4) improve encoding format
5) try using less classes (most important ones, so that less classes but more balanced dataset)
6) adding more files
7) adjusting hyperparameters such as learning rate, dropout, ...etc
8) splitting train, val, test
9) using equal percentages of each class (adjusting splitting)